In [8]:
import warnings 
import pennylane as qml
import pennylane.numpy as np
from scipy.optimize import minimize

warnings.filterwarnings('ignore') 

In [2]:
dev = qml.device('qiskit.aer', wires=2, shots=1000)

In [27]:
@qml.qnode(dev)
def circuit(weights, train=True):
    """
        this circuit is meant to be optimized to return only bit strings that contain a single 
        value one.
    """

    for qubit,weight in enumerate(weights):
        qml.RY(weight, wires=qubit)

    if(train):
        H = 0.5*(qml.PauliZ(0) @ qml.PauliZ(1)) # the ising hamiltonian for our problem
        return qml.expval(H)
    
    return qml.counts()


In [9]:
x0 = np.random.uniform(0, np.pi, size=2, requires_grad=True)

In [10]:
minimized = minimize(circuit, x0=x0, method="COBYLA")
minimized

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.498
       x: [ 3.087e+00  2.332e-02]
    nfev: 30
   maxcv: 0.0

In [26]:
# the trained model returns, almost, always the bit strings we were expecting it to
results = circuit(minimized.x, train=False)

for bit_string, total in results.items():
    print(f"{bit_string} appeared {total} times")

00 appeared 1 times
10 appeared 999 times
